<a href="https://colab.research.google.com/github/jasminajovanovic/Machine-Learning/blob/master/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier





# Read the CSV and Perform Basic Data Cleaning

In [3]:
df = pd.read_csv("cumulative.csv")
df = df.drop(columns=["rowid", "kepid", "kepoi_name", "kepler_name", "koi_pdisposition", "koi_score", "koi_tce_delivname"])
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,615.8,19.5,-19.5,2.26,0.26,-0.15,793.0,93.59,29.45,-16.65,35.8,1.0,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443.0,9.11,2.87,-1.62,25.8,2.0,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,FALSE POSITIVE,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638.0,39.30,31.04,-10.49,76.3,1.0,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395.0,891.96,668.95,-230.35,505.6,1.0,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,CONFIRMED,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406.0,926.16,874.33,-314.24,40.9,1.0,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


# Create a Train Test Split

Use `koi_disposition` for the y values

In [0]:
y = df["koi_disposition"]

In [5]:
y.shape

(8744,)

In [0]:
y_targets = y.unique()

In [0]:
X = df.drop(columns="koi_disposition")

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [9]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
8017,0,1,1,0,0.806277,4.947000e-06,-4.947000e-06,131.785670,0.006720,-0.006720,0.697,0.260,-0.451,7.27500,0.31200,-0.31200,70.8,3.8,-3.8,0.89,0.29,-0.09,2046.0,4119.09,3927.86,-1221.11,37.9,1.0,6122.0,165.0,-184.0,4.471,0.054,-0.229,0.996,0.324,-0.108,290.81723,38.539120,13.614
1233,0,1,1,0,3.582077,4.318000e-06,-4.318000e-06,355.515064,0.000864,-0.000864,0.966,0.011,-0.008,4.42280,0.06360,-0.06360,1534.4,17.7,-17.7,5.33,1.52,-0.70,1265.0,604.27,524.99,-215.04,138.4,1.0,6109.0,192.0,-235.0,4.422,0.090,-0.195,0.993,0.283,-0.131,296.07822,43.136940,15.193
2592,0,0,0,0,5.060923,2.616000e-05,-2.616000e-05,134.473160,0.004730,-0.004730,0.944,0.006,-0.643,2.58600,0.11400,-0.11400,71.6,5.3,-5.3,0.98,0.13,-0.06,998.0,234.93,88.75,-45.74,15.7,1.0,5637.0,101.0,-112.0,4.492,0.048,-0.112,0.911,0.121,-0.060,289.91742,40.828606,13.346
4770,0,1,0,1,8.480304,3.320000e-07,-3.320000e-07,135.854534,0.000031,-0.000031,0.235,0.003,-0.003,7.46159,0.00234,-0.00234,83115.0,18.9,-18.9,65.00,11.03,-16.56,1504.0,1210.22,566.98,-600.16,5219.5,1.0,7059.0,126.0,-169.0,3.946,0.195,-0.105,2.210,0.375,-0.563,298.80020,46.665539,7.631
6632,0,0,0,1,4.994716,4.495000e-05,-4.495000e-05,136.183300,0.009500,-0.009500,0.698,0.277,-0.462,3.44000,0.25700,-0.25700,53.0,4.6,-4.6,2.20,0.35,-0.83,1861.0,2836.06,1425.22,-1863.86,14.1,1.0,6409.0,174.0,-194.0,3.706,0.320,-0.080,2.830,0.458,-1.068,282.58215,46.815510,13.352


In [0]:
X_features = X.columns

In [11]:
y.shape

(8744,)

# Pre-processing

Scale the data using the MinMaxScaler

In [0]:
X_minmax = MinMaxScaler().fit(X_train)

X_train = X_minmax.transform(X_train)
X_test = X_minmax.transform(X_test)


# Train the Support Vector Machine

In [13]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [14]:
print(f"Training Data Score: {model.score(X_train, y_train)}")
print(f"Testing Data Score: {model.score(X_test, y_test)}")

Training Data Score: 0.8502592253735896
Testing Data Score: 0.838975297346752


In [16]:
# from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.73      0.56      0.63       523
     CONFIRMED       0.68      0.80      0.73       594
FALSE POSITIVE       0.98      1.00      0.99      1069

      accuracy                           0.84      2186
     macro avg       0.80      0.78      0.79      2186
  weighted avg       0.84      0.84      0.83      2186



# Hyperparameter Tuning

Use `GridSearchCV` to tune the `C` and `gamma` parameters

In [17]:
model

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [0]:
# Create the GridSearchCV model
# from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [0]:
# Train the model with GridSearch
grid.fit(X_train, y_train)

In [34]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 50, 'gamma': 0.0001}
0.8824336688014639


In [19]:
# classifier = LogisticRegression(C=50, penalty='l1')

model = SVC(kernel='linear', C=50, gamma=0.0001)
model

SVC(C=50, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [21]:
model.fit(X_train, y_train)

SVC(C=50, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [22]:
print(f"Training Data Score: {model.score(X_train, y_train)}")
print(f"Testing Data Score: {model.score(X_test, y_test)}")

Training Data Score: 0.8877706617871303
Testing Data Score: 0.8810612991765783


In [23]:
# from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.85      0.65      0.73       523
     CONFIRMED       0.75      0.87      0.81       594
FALSE POSITIVE       0.98      1.00      0.99      1069

      accuracy                           0.88      2186
     macro avg       0.86      0.84      0.84      2186
  weighted avg       0.88      0.88      0.88      2186



# Random Forst Classifier


In [35]:
# from sklearn.ensemble import RandomForestClassifier
# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)
# predictions = rf.predict(X_test)

0.8952424519670631

In [0]:
importances = rf.feature_importances_
# importances

In [37]:
sorted(zip(rf.feature_importances_, X_features), reverse=True)

[(0.10994198467579953, 'koi_fpflag_co'),
 (0.09618727406272261, 'koi_fpflag_nt'),
 (0.06747660530396, 'koi_fpflag_ss'),
 (0.05713551890118187, 'koi_model_snr'),
 (0.048895872633851624, 'koi_prad'),
 (0.036701711747002964, 'koi_fpflag_ec'),
 (0.03400737355251167, 'koi_prad_err2'),
 (0.031670782114020045, 'koi_duration_err2'),
 (0.0313778323334402, 'koi_steff_err2'),
 (0.03097501244184158, 'koi_duration_err1'),
 (0.029890169403237774, 'koi_prad_err1'),
 (0.027255960270498425, 'koi_steff_err1'),
 (0.025780488303201954, 'koi_time0bk_err1'),
 (0.024921558092052914, 'koi_duration'),
 (0.022835501541166984, 'koi_period'),
 (0.020985071271275097, 'koi_time0bk_err2'),
 (0.020420492230208007, 'koi_period_err2'),
 (0.019489607254098002, 'koi_depth'),
 (0.01844621295902018, 'koi_impact'),
 (0.018356690666946323, 'koi_period_err1'),
 (0.016949268836040088, 'koi_insol_err1'),
 (0.016835789643797873, 'koi_teq'),
 (0.014854160907590461, 'koi_depth_err1'),
 (0.014428234960682181, 'koi_time0bk'),
 (0.01

In [38]:
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.73      0.56      0.63       523
     CONFIRMED       0.68      0.80      0.73       594
FALSE POSITIVE       0.98      1.00      0.99      1069

      accuracy                           0.84      2186
     macro avg       0.80      0.78      0.79      2186
  weighted avg       0.84      0.84      0.83      2186



# Tune Random Forest Classifier


In [39]:
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators=1000)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.8947849954254345

# Logistic Regression Model



In [40]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [41]:
classifier.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
predictions = classifier.predict(X_test)

In [43]:
classifier.score(X_test, y_test)

0.833028362305581

In [44]:
print(classification_report(y_test, predictions))


                precision    recall  f1-score   support

     CANDIDATE       0.70      0.58      0.63       523
     CONFIRMED       0.68      0.76      0.72       594
FALSE POSITIVE       0.98      1.00      0.99      1069

      accuracy                           0.83      2186
     macro avg       0.79      0.78      0.78      2186
  weighted avg       0.83      0.83      0.83      2186



In [45]:
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
# Create the GridSearchCV model
# from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'penalty': ['l2', 'l1']}
grid = GridSearchCV(classifier, param_grid, verbose=3)
grid.fit(X_train, y_train)


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] C=1, penalty=l2 .................................................
[CV] ..................... C=1, penalty=l2, score=0.839, total=   0.1s
[CV] C=1, penalty=l2 .................................................


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify 

[CV] ..................... C=1, penalty=l2, score=0.836, total=   0.1s
[CV] C=1, penalty=l2 .................................................
[CV] ..................... C=1, penalty=l2, score=0.849, total=   0.1s
[CV] C=1, penalty=l1 .................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] ..................... C=1, penalty=l1, score=0.870, total=   0.7s
[CV] C=1, penalty=l1 .................................................


...

[CV] .................... C=50, penalty=l1, score=0.869, total=  12.3s
[CV] C=50, penalty=l1 ................................................


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[CV] .................... C=50, penalty=l1, score=0.898, total=  19.5s


[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:  1.3min finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1, 5, 10, 50], 'penalty': ['l2', 'l1']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [47]:
print(grid.best_params_)
print(grid.best_score_)


{'C': 50, 'penalty': 'l1'}
0.8819762122598354


In [48]:
classifier = LogisticRegression(C=50, penalty='l1')
classifier

LogisticRegression(C=50, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
classifier.fit(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8764867337602927

In [50]:
classifier.score(X_test, y_test)

0.8764867337602927